In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.svm import SVC
import tensorflow as tf

In [0]:
#Importing Data
data = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')
y = pd.read_csv('labels.csv')['status_group'].values

In [0]:
#drop funder,subvillage,wpt_name,ward,installer,recorded_by,scheme_name,extraction_type_group,extraction_type_class,waterpoint_type_group,management_group,payment_type,quality_group,quantity_group,source_type,source_class,id
#one-hot basin,region, lga,extraction_type,management,water_quality,quantity,source,waterpoint_type
#label-encode  public_meeting,scheme_management,permit,payment
#Impute mode public_meeting,permit,scheme_management

In [0]:
#drop unecessary columns
data = data.drop(columns=['lga','date_recorded','funder','subvillage','wpt_name','ward','installer','recorded_by','scheme_name','extraction_type_group','extraction_type_class','waterpoint_type_group','management_group','payment_type','quality_group','quantity_group','source_type','source_class','id'])
data_test = data_test.drop(columns=['lga','date_recorded','funder','subvillage','wpt_name','ward','installer','recorded_by','scheme_name','extraction_type_group','extraction_type_class','waterpoint_type_group','management_group','payment_type','quality_group','quantity_group','source_type','source_class','id'])

In [0]:
#impute nan values
data['public_meeting'] = data['public_meeting'].fillna(data['public_meeting'].mode()[0])
data['permit'] = data['permit'].fillna(data['permit'].mode()[0])
data['scheme_management'] = data['scheme_management'].fillna(data['scheme_management'].mode()[0])

data_test['public_meeting'] = data_test['public_meeting'].fillna(data_test['public_meeting'].mode()[0])
data_test['permit'] = data_test['permit'].fillna(data_test['permit'].mode()[0])
data_test['scheme_management'] = data_test['scheme_management'].fillna(data_test['scheme_management'].mode()[0])

In [0]:
#label encode ordinal features
label_encoder_feature = ['public_meeting','scheme_management','permit','payment']
for i in label_encoder_feature :
  labelencoder = LabelEncoder()
  labelencoder.fit(data[i])
  data[i] = labelencoder.transform(data[i])
  data_test[i] = labelencoder.transform(data_test[i])

In [0]:
#one hot encode nominal features
arr = ['basin',13,40,57,69,76,80,89]
for i in arr :
  print(i)
  ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[i])],remainder='passthrough')
  data = ct.fit_transform(data)
  data_test = ct.transform(data_test)
  data = pd.DataFrame(data)
  data_test = pd.DataFrame(data_test)
  data = data.drop(columns=[0],axis=1)
  data_test = data_test.drop(columns=[0],axis=1)
  data = data.values
  data_test = data_test.values
  data = pd.DataFrame(data)
  data_test = pd.DataFrame(data_test)

In [42]:
#label encode target variable
label_encoder_target = LabelEncoder()
y = label_encoder_target.fit_transform(y)
label_encoder_target.classes_

array(['functional', 'functional needs repair', 'non functional'],
      dtype=object)

In [0]:
#converted into arrays
x = data.values
x_test = data_test.values

In [0]:
#scaling the features
sc = StandardScaler()
x = sc.fit_transform(x)
x_test = sc.transform(x_test)

In [0]:
#used xg boost
model = XGBClassifier()
model.fit(x, y)

In [13]:
#used non-linear svm 
svm_model = SVC(kernel='rbf')
svm_model.fit(x,y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
#used ANN
y = tf.keras.utils.to_categorical(y)
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(128,kernel_initializer='uniform',activation='relu',input_dim=95))
model.add(tf.keras.layers.Dense(128,kernel_initializer='uniform',activation='relu'))
model.add(tf.keras.layers.Dense(3,kernel_initializer='uniform',activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x,y,epochs=500,batch_size=32)

In [0]:
#for svm and xg boost
y_pred = model.predict(x_test)
pred = label_encoder_target.inverse_transform(pred)
sub = pd.read_csv('SubmissionFormat.csv')
sub['status_group'] = pred
sub.to_csv('finalss.csv',index=False)

In [0]:
#for ann
y_pred = model.predict(x_test)
pred = []
for i in y_pred :
  pred.append(np.argmax(i))
pred = label_encoder_target.inverse_transform(pred)
sub = pd.read_csv('SubmissionFormat.csv')
sub['status_group'] = pred
sub.to_csv('finalss.csv',index=False)

In [0]:
'''
XG Boost 0.7479
SVM 0.7698
ANN 0.7763
'''